In [8]:
import ipywidgets as widgets
from IPython.display import display, HTML
import time
import numpy as np

output_area = widgets.Output()

def set_custom_output(text):
    with output_area:
        output_area.clear_output()
        print(text)

def create_selectable_array(label, options, trim_projections, default_selected=None):
    dropdown = widgets.Dropdown(
        options=options,
        description=f'Select {label}:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='auto', max_width='300px')
    )

    add_button = widgets.Button(
        description='Add',
        button_style='primary',
        layout=widgets.Layout(width='auto', margin='0 0 0 5px')
    )

    array_box = widgets.HBox([])
    added_items = set()

    # Check if default_selected is a list and add those items
    if isinstance(default_selected, list):
        for selected in default_selected:
            if selected in options:
                added_items.add(selected)
                item_label = widgets.Label(selected)
                remove_button = widgets.Button(
                    description='x',
                    button_style='danger',
                    layout=widgets.Layout(width='auto', margin='0 10px 0 10px')
                )

                def remove_item(btn, selected_value=selected):  # Capture selected_value
                    if selected_value in added_items:
                        added_items.remove(selected_value)
                        array_box.children = [child for child in array_box.children if child.children[0].value != selected_value]
                    trim_projections.disabled = ("Notional" not in added_items)
                    if trim_projections.disabled:
                        trim_projections.value = False
                remove_button.on_click(remove_item)
                item_box = widgets.HBox([item_label, remove_button])
                array_box.children += (item_box,)

    def add_item(b):
        if dropdown.value and dropdown.value not in added_items:
            added_items.add(dropdown.value)
            item_label = widgets.Label(dropdown.value)
            remove_button = widgets.Button(
                description='x',
                button_style='danger',
                layout=widgets.Layout(width='auto', margin='0 10px 0 10px')
            )

            def remove_item(btn, selected_value=dropdown.value):  # Capture selected_value
                if selected_value in added_items:
                    added_items.remove(selected_value)
                    array_box.children = [child for child in array_box.children if child.children[0].value != selected_value]
                trim_projections.disabled = ("Notional" not in added_items)
                if trim_projections.disabled:
                    trim_projections.value = False
                
            remove_button.on_click(remove_item)
            item_box = widgets.HBox([item_label, remove_button])
            array_box.children += (item_box,)
        
        trim_projections.disabled = ("Notional" not in added_items)
        if trim_projections.disabled:
            trim_projections.value = False

    add_button.on_click(add_item)

    dropdown_row = widgets.HBox([dropdown, add_button], layout=widgets.Layout(margin='5px 0px', align_items='center'))
    return widgets.VBox([dropdown_row, array_box], layout=widgets.Layout(margin='5px 0px'), align_items='center', flex_flow='row wrap', font_size="16px")

def create_pack_config(remove_button_callback, default_pack_type='Group'):
    global_header = widgets.HTML("<b>Global Inputs</b>")
    
    # Mapping of display names to NumPy date values
    as_of_date_options = {
        '2024-01-01': np.datetime64('2024-01-01'),
        '2024-02-01': np.datetime64('2024-02-01'),
        '2024-03-01': np.datetime64('2024-03-01')
    }
    
    as_of_date = widgets.Dropdown(
        options=list(as_of_date_options.keys()),
        description='As of Date:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='auto', max_width='200px')
    )
    
    pack_type = widgets.Dropdown(
        options=['Group', 'Channel', 'Entity'],
        description='Pack Type:',
        style={'description_width': 'initial'},
        value=default_pack_type,
        layout=widgets.Layout(width='auto', max_width='200px')
    )
    
    trim_projections = widgets.Checkbox(
        value=True,
        description='Trim Projections',
        style={'description_width': 'initial'}
    )

    entities = create_selectable_array('Entity', ['Entity A', 'Entity B', 'Entity C'], trim_projections, default_selected= ['Entity A'])
    entities.layout.display = 'none'
    
    metrics = create_selectable_array('Metric', ['Notional', 'Interest', 'Yield'], trim_projections, default_selected= ['Notional'])
    bt_months = create_selectable_array('BT Month', [str(i) for i in range(1, 13)], trim_projections, default_selected=['1', '3', '6', '12'])
    
    include_active = widgets.Checkbox(
        value=(default_pack_type in ['Channel', 'Entity']),
        description='Include Only Active Channels',
        style={'description_width': 'initial'}
    )
    
    def update_fields(change):
        if pack_type.value == 'Entity':
            entities.layout.display = 'block'
            include_active.value = True
            include_sensitivity.value = False  # Uncheck the checkbox
            include_sensitivity.disabled = True  # Disable the checkbox
        else:
            entities.layout.display = 'none'
            include_sensitivity.value = True
            include_active.value = (pack_type.value == 'Channel')
            include_sensitivity.disabled = False  # Enable the checkbox if not 'Entity'
     
    pack_type.observe(update_fields, names='value')
    
    plot_header = widgets.HTML("<b>Plot Inputs</b>")
    
    rate_scenario = widgets.Dropdown(
        options=['Scenario 1', 'Scenario 2', 'Scenario 3'],
        description='Rate Scenario:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='auto', max_width='200px')
    )
    
    table_header = widgets.HTML("<b>Table Inputs</b>")
    rate_shock_scenario = widgets.Dropdown(
        options=['Shock 1', 'Shock 2', 'Shock 3'],
        description='Rate Shock Scenario:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='auto', max_width='300px')  # Increase max_width
    )
    
    include_sensitivity = widgets.Checkbox(
        value=(pack_type.value != 'Entity'),
        description='Include Sensitivity Cols',
        style={'description_width': 'initial'},
        disabled=(pack_type.value == 'Entity')
    )
    
    remove_button = widgets.Button(
        description='x',
        button_style='danger',
        layout=widgets.Layout(width='30px', height='30px', margin='10px 0 10px 0')  # Flush with box
    )
    remove_button.on_click(remove_button_callback)

    global_inputs = widgets.VBox([global_header, as_of_date, pack_type, entities, metrics, bt_months, include_active])
    plot_inputs = widgets.VBox([plot_header, rate_scenario, trim_projections])
    table_inputs = widgets.VBox([table_header, rate_shock_scenario, include_sensitivity])
    
    return widgets.HBox([widgets.VBox([global_inputs, plot_inputs, table_inputs], 
                                       layout=widgets.Layout(border='1px solid gray', padding='10px', margin='10px 0px', width='auto')),
                         remove_button])

pack_configs = []
add_button = widgets.Button(
    description='Add Pack Configuration',
    icon='plus',
    button_style='info',
    layout=widgets.Layout(width='320px')
)

form_container = widgets.VBox()
run_button = widgets.Button(
    description='Run',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='320px', margin='10px 0px')
)

def add_new_pack_config(b):
    def remove_pack_config(change):
        pack_configs.remove(new_pack_config)
        form_container.children = pack_configs

    new_pack_config = create_pack_config(remove_pack_config)
    pack_configs.append(new_pack_config)
    form_container.children = pack_configs

add_new_pack_config(None)
add_button.on_click(add_new_pack_config)

def set_form_enabled(enabled):
    for config in pack_configs:
        for widget in config.children[0].children:
            widget.disabled = not enabled
        config.children[1].disabled = not enabled

    add_button.disabled = not enabled
    run_button.disabled = not enabled

def collect_form_values():
    form_values = []
    for config in pack_configs:
        # Access the correct widgets using their indices
        global_inputs = config.children[0].children
        
        entities = [child.children[0].value for child in global_inputs[3].children[1].children if isinstance(child, widgets.HBox)]
        metrics = [child.children[0].value for child in global_inputs[4].children[1].children if isinstance(child, widgets.HBox)]
        bt_months = [child.children[0].value for child in global_inputs[5].children[1].children if isinstance(child, widgets.HBox)]

        # Check for at least one item selected in each multiple select box
        if not entities or not metrics or not bt_months:
            set_custom_output("Error: Each multiple select box must have at least one item selected.")
            return []

        pack_value = {
            'pack_type': global_inputs[1].value,  # Pack Type
            'as_of_date': as_of_date_options[global_inputs[0].value],  # As of Date
            'entities': entities,  # Entities
            'metrics': metrics,  # Metrics
            'bt_months': bt_months,  # BT Months
            'include_active': global_inputs[5].value  # Include Active
        }
        form_values.append(pack_value)
    return form_values

def run_function(b):
    set_form_enabled(False)
    set_custom_output("Running... Please wait.")
    time.sleep(3)
    values = collect_form_values()
    if values:  # Only show if collection was successful
        set_custom_output(f"Run completed. Collected values: {values}")
    set_form_enabled(True)

run_button.on_click(run_function)

title = widgets.HTML("<h3>Pack Configuration Form</h3>")
display(widgets.VBox([title, form_container, add_button, run_button, output_area]))

display(HTML('''
<style>
    b {
        font-size: 18px;
    }
</style>
'''))

# notional label bug!!!

In [11]:
import re
input_string = str(pack_configs)

In [12]:
pattern = r"Label\(value='([^']+)'\).*?Button\(button_style='danger'"
matches = re.findall(pattern, input_string, re.DOTALL)
matches

['Entity A', 'Notional', 'Interest', '1', '3', '6', '12']

In [13]:
new_str = input_string[input_string.find("HBox")+1:]
matches = re.findall(pattern, new_str, re.DOTALL)
matches

['Entity A', 'Notional', 'Interest', '1', '3', '6', '12']

In [14]:
input_string.find("HBox")

1

In [15]:
Test = input_string.split("Dropdown")
matches = re.findall(pattern, Test[5], re.DOTALL)
matches

['1', '3', '6', '12']

In [16]:
pattern = r"value='([^']+)'|Checkbox\(value=(True|False),"
matches = re.findall(pattern, input_string, re.DOTALL)
matches = [match for match in matches if "<b>" not in match]
matches

[('<b>Global Inputs</b>', ''),
 ('2024-01-01', ''),
 ('Group', ''),
 ('Entity A', ''),
 ('Entity A', ''),
 ('Interest', ''),
 ('Notional', ''),
 ('Interest', ''),
 ('1', ''),
 ('1', ''),
 ('3', ''),
 ('6', ''),
 ('12', ''),
 ('', 'False'),
 ('<b>Plot Inputs</b>', ''),
 ('Scenario 1', ''),
 ('', 'True'),
 ('<b>Table Inputs</b>', ''),
 ('Shock 1', ''),
 ('', 'True')]

In [17]:
import re

# Define the regex pattern
pattern = r"value='([^']+)'|Checkbox\(value=(True|False),"

# Find all matches in the input string
matches = re.findall(pattern, input_string)

# Create a flat list of matches
flat_matches = [match for group in matches for match in group if (match and "<b>" not in match)]

# Print the flat matches
print(flat_matches)

['2024-01-01', 'Group', 'Entity A', 'Entity A', 'Interest', 'Notional', 'Interest', '1', '1', '3', '6', '12', 'False', 'Scenario 1', 'True', 'Shock 1', 'True']


In [18]:
input_string

"[HBox(children=(VBox(children=(VBox(children=(HTML(value='<b>Global Inputs</b>'), Dropdown(description='As of Date:', layout=Layout(max_width='200px', width='auto'), options=('2024-01-01', '2024-02-01', '2024-03-01'), style=DescriptionStyle(description_width='initial'), value='2024-01-01'), Dropdown(description='Pack Type:', layout=Layout(max_width='200px', width='auto'), options=('Group', 'Channel', 'Entity'), style=DescriptionStyle(description_width='initial'), value='Group'), VBox(children=(HBox(children=(Dropdown(description='Select Entity:', layout=Layout(max_width='300px', width='auto'), options=('Entity A', 'Entity B', 'Entity C'), style=DescriptionStyle(description_width='initial'), value='Entity A'), Button(button_style='primary', description='Add', layout=Layout(margin='0 0 0 5px', width='auto'), style=ButtonStyle())), layout=Layout(align_items='center', margin='5px 0px')), HBox(children=(HBox(children=(Label(value='Entity A'), Button(button_style='danger', description='x', 

In [27]:
import re

# Regular expression pattern
pattern = r"value='([^']+)'|Checkbox\(value=(True|False)\)"

# Find all matches
matches = re.findall(pattern, input_string)

# Flatten the list of tuples into a flat list
flat_matches = [match for group in matches for match in group if match and "<b>" not in match]

print(flat_matches)

['2024-01-01', 'Group', 'Entity A', 'Entity A', 'Interest', 'Notional', 'Interest', '1', '1', '3', '6', '12', 'Scenario 1', 'Shock 1']
